In [9]:
from logic import *

class ALU():
    def __init__(self, name, zeroFlag = 1, overflowFlag = 0, negativeFlag = 0):
        self.name = name
        self.zeroFlag = zeroFlag
        self.overflowFlag  = overflowFlag
        self.negativeFlag= negativeFlag

    @staticmethod
    def toString(byte):
        s = ""
        for bit in byte:
            x = str(int(bit))
            s += x
        return s
    
    def isZero(self, byte):
        Out = Not("Out")
        def isSimpleZero(byte):
            if len(byte) == 2:
                F0 = Or("F0")
                F0.A.set(bit(byte, 0))
                F0.B.set(bit(byte, 1))
                return F0.C.value
            else:
                F1 = Or("F1")
                left = isSimpleZero(byte[0:int(len(byte)/2)])
                right = isSimpleZero(byte[int(len(byte)/2):int(len(byte))])
                F1.A.set(left)
                F1.B.set(right)
                # print(left, right)
                return (F1.C.value)
        Out.A.set(isSimpleZero(byte))
        Out.evaluate()
        flag = str(int(Out.B.value))
        self.zeroFlag = flag
        return flag
    
    def isNegative(self, byte):
        F0 = And("F0")
        F0.A.set(bit(byte, 0))
        F0.B.set(True)
        flag = str(int(F0.C.value))
        self.negativeFlag = flag
        return(flag)
    


In [12]:
import unittest

In [ ]:
class TestIsZero(unittest.TestCase):
    test = ALU("isZero")
    def test1(self):
        self.assertEqual(self.test.isZero('11000000'),'0'), "Should be False"
    def test2(self):
        self.assertEqual(self.test.isZero('01000010'),'0'), "Should be False"
    def test3(self):
        self.assertEqual(self.test.isZero('11000010'), '0'), "Should be False"
    def test4(self):
        self.assertEqual(self.test.isZero('11111111'), '0'), "Should be False"
    def test5(self):
        self.assertEqual(self.test.isZero('00000000'),'1'), "Should be True"

unittest.main(argv=[''], verbosity=2, exit=False)

In [ ]:
class TestIsNegative(unittest.TestCase):
    test = ALU("isNegative")
    def test1(self):
        self.assertEqual(self.test.isNegative('11000000'),'1'), "Should be True"
    def test2(self):
        self.assertEqual(self.test.isNegative('01000010'),'0'), "Should be False"
    def test3(self):
        self.assertEqual(self.test.isNegative('11000010'), '1'), "Should be True"
    def test4(self):
        self.assertEqual(self.test.isNegative('11111111'), '1'), "Should be True"
    def test5(self):
        self.assertEqual(self.test.isNegative('00000000'),'0'), "Should be False"

unittest.main(argv=[''], verbosity=2, exit=False)